In [79]:
import requests
from time import sleep

def get_token():
    headers = {
        'authority': 'clerk.suno.com',
        'accept': '*/*',
        'accept-language': 'en-BR,en;q=0.9,pt-BR;q=0.8,pt;q=0.7,es-ES;q=0.6,es;q=0.5,de-LI;q=0.4,de;q=0.3,en-GB;q=0.2,en-US;q=0.1,sk;q=0.1',
        'content-type': 'application/x-www-form-urlencoded',
        #grab it from the network tab
        #'cookie': '',
        'origin': 'https://suno.com',
        'referer': 'https://suno.com',
        'sec-ch-ua': '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
    }
    #Adquire this from your logged-in session on Suno
    #response = requests.post('https://clerk.suno.com/v1/client/sessions/...', headers=headers)
    print(response.text)
    print(response)
    token = response.json()['response']['last_active_token']['jwt']
    return token

In [96]:
def get_model_id(general_prompt="", lyrics="", style="", instrumental=False):
    token = get_token()
    url = "https://studio-api.prod.suno.com/api/generate/v2/"

    headers = {
        "authority": "studio-api.prod.suno.com",
        "accept": "*/*",
        "accept-language": "en-BR,en;q=0.9,pt-BR;q=0.8,pt;q=0.7,es-ES;q=0.6,es;q=0.5,de-LI;q=0.4,de;q=0.3,en-GB;q=0.2,en-US;q=0.1,sk;q=0.1",
        "authorization": f"Bearer {token}",
        "content-type": "text/plain;charset=UTF-8",
        "origin": "https://suno.com",
        "referer": "https://suno.com",
        "sec-ch-ua": '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"macOS"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    }
    #if(lyrics):
    data = f'{{"mv":"chirp-v4","prompt":"{lyrics}","tags":"{style}","make_instrumental":{str(instrumental).lower()}}}'
    #else:
    #    data = f'{{"mv":"chirp-v4","prompt":"{lyrics}","tags":"{style}","make_instrumental":{str(instrumental).lower()}}}'
    
    print(data)
    response = requests.post(url, headers=headers, data=data)
    task_response = response.json()
    clip_ids = []
    for clip in task_response["clips"]:
        clip_ids.append(clip["id"])
    return clip_ids

In [97]:
def get_feed(clip_ids):

    def match_results(clip_ids, feed_response):
        result_data = []
        for clip_id in clip_ids:
            for model_in_feed in feed_response:
                if(model_in_feed["id"] == clip_id):
                    if(model_in_feed["audio_url"]):
                        result_data.append(model_in_feed)
        return result_data if len(result_data) == 2 else False

    token = get_token()
    headers = {
        'authority': 'studio-api.prod.suno.com',
        'accept': '*/*',
        'accept-language': 'en-BR,en;q=0.9,pt-BR;q=0.8,pt;q=0.7,es-ES;q=0.6,es;q=0.5,de-LI;q=0.4,de;q=0.3,en-GB;q=0.2,en-US;q=0.1,sk;q=0.1',
        'authorization': f'Bearer {token}',
        'origin': 'https://suno.com',
        'referer': 'https://suno.com/',
        'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    }

    response = requests.get('https://studio-api.prod.suno.com/api/feed/', headers=headers)
    
    feed_response = response.json()
    return match_results(clip_ids, feed_response)

In [98]:

def get_suno_songs(prompt="", instrumental=False, lyrics="", style=""):
    clip_ids = get_model_id(prompt, lyrics, style, instrumental)
    counter = 0

    while True:
        matched_results = get_feed(clip_ids)
        if(matched_results):
            break
        else:
            counter+=1
            if(counter > 250):
                print("Aborted after 250s")
                break
            sleep(1)

    audio_urls = []
    for result in matched_results:
        audio_urls.append(result["audio_url"])
    return audio_urls

In [100]:
lyrics = '''fuck the artificial intelligence'''
get_suno_songs(style="8-bit chipmunk dogs", lyrics=lyrics)

{"response":{"object":"session","id":"sess_2p1oaq9ieiUTHIlEpgFTzCKS9ZA","status":"active","expire_at":1739719008364,"abandon_at":1739719008364,"last_active_at":1733012022892,"last_active_organization_id":null,"actor":null,"user":{"id":"user_2d6oP6jzxz6hkbxuqP9ruTsF03G","object":"user","username":null,"first_name":"João Paulo","last_name":"Apolinário Passos","image_url":"https://img.clerk.com/eyJ0eXBlIjoicHJveHkiLCJzcmMiOiJodHRwczovL2ltYWdlcy5jbGVyay5kZXYvb2F1dGhfZ29vZ2xlL2ltZ18yZDZvUDROQlREdjRMVUt6OXcxRGNFM2FCcTIifQ","has_image":true,"primary_email_address_id":"idn_2d6oP71PA0nV9hk4iZUV7LT9J7s","primary_phone_number_id":null,"primary_web3_wallet_id":null,"password_enabled":false,"two_factor_enabled":false,"totp_enabled":false,"backup_code_enabled":false,"email_addresses":[{"id":"idn_2d6oP71PA0nV9hk4iZUV7LT9J7s","object":"email_address","email_address":"joaopaulo.passos@gmail.com","reserved":false,"verification":{"status":"verified","strategy":"from_oauth_google","attempts":null,"expire_

['https://audiopipe.suno.ai/?item_id=ac06c8ab-e510-45be-8e74-4eea8792e89f',
 'https://audiopipe.suno.ai/?item_id=c0ba9df6-1ad4-42be-883f-4f63a39363fe']